# **Velour Dataset Integrations**

Velour is crafted to seamlessly integrate with your existing workflows.

Our client is equipped to handle a wide array of tasks, including classification, object detection, and semantic segmentation.

When it comes to integration, think of it as creating a dataloader. The Velour datasets are structured in a hierarchy of standard types that abstract into the datatypes you provide. 

At the top level, you'll find `velour.Dataset` which contains a list of `velour.GroundTruth`, which, in turn, consist of `velour.Datum` and `velour.Annotation`.

The ultimate objective is to effortlessly map all available metadata into Velour, empowering you to utilize it later for precise evaluations and effective stratification.

In [1]:
import json
from pathlib import Path

from velour import (
    Client,
    Dataset,
    Model,
    Datum,
    Annotation,
    GroundTruth, 
    Prediction,
    Label,
)
from velour.schemas import (
    BoundingBox, 
    Polygon, 
    BasicPolygon, 
    Point,
)
from velour.enums import TaskType

In [2]:
client = Client("http://0.0.0.0:8000")

Succesfully connected to http://0.0.0.0:8000/.


# **Creating a Dataset**

In [3]:
dataset = Dataset(
    client=client,   
    name="myDataset",
    metadata={        # optional, metadata can take `str`, `int`, `float` value types.
        "some_string": "hello_world",
        "some_number": 1234,
        "a_different_number": 1.234,
    },
    geospatial=None,  # optional, define a GeoJSON
    reset=True,
)

# **Image Classification**

In [4]:
image_classifications = [
    {"path": "a/b/c/img1.png", "annotations": [{"class_label": "dog"}]},
    {"path": "a/b/c/img2.png", "annotations": [{"class_label": "cat"}]}
]

In [5]:
def create_groundtruth_from_image_classification_dict(element: dict):
    
    # create Datum using filename, save the full filepath into metadata
    datum = Datum(
        uid=Path(element["path"]).stem,
        metadata={
            "path": element["path"]
        }
    )

    # create Annotation
    annotations = [
        Annotation(
            task_type=TaskType.CLASSIFICATION,
            labels=[
                Label(key=key, value=value)
                for label in element["annotations"]
                for key, value in label.items()
            ]
        )
    ]

    # create and return GroundTruth
    return GroundTruth(
        datum=datum,
        annotations=annotations,
    )

In [6]:
for element in image_classifications:
    # create groundtruth
    groundtruth = create_groundtruth_from_image_classification_dict(element)

    # add groundtruth to dataset
    dataset.add_groundtruth(groundtruth)
    
    print(groundtruth)

{'datum': {'dataset': 'myDataset', 'uid': 'img1', 'metadata': {'path': 'a/b/c/img1.png'}, 'geospatial': {}}, 'annotations': [{'task_type': 'classification', 'labels': [{'key': 'class_label', 'value': 'dog', 'score': None}], 'metadata': {}, 'bounding_box': None, 'polygon': None, 'multipolygon': None, 'raster': None, 'jsonb': None}]}
{'datum': {'dataset': 'myDataset', 'uid': 'img2', 'metadata': {'path': 'a/b/c/img2.png'}, 'geospatial': {}}, 'annotations': [{'task_type': 'classification', 'labels': [{'key': 'class_label', 'value': 'cat', 'score': None}], 'metadata': {}, 'bounding_box': None, 'polygon': None, 'multipolygon': None, 'raster': None, 'jsonb': None}]}


# **Image Object Detection**

In [7]:
object_detections = [
    {"path": "a/b/c/img3.png", "annotations": [{"class_label": "dog", "bbox": {"xmin": 16, "ymin": 130, "xmax": 70, "ymax": 150}}, {"class_label": "person", "bbox": {"xmin": 89, "ymin": 10, "xmax": 97, "ymax": 110}}]},
    {"path": "a/b/c/img4.png", "annotations": [{"class_label": "cat", "bbox": {"xmin": 500, "ymin": 220, "xmax": 530, "ymax": 260}}]},
    {"path": "a/b/c/img5.png", "annotations": []}
]

In [8]:
def create_groundtruth_from_object_detection_dict(element: dict):
    
    # create Datum using filename, save the full filepath into metadata
    datum = Datum(
        uid=Path(element["path"]).stem,
        metadata={
            "path": element["path"] 
        }
    )

    # create Annotations
    annotations = [
        Annotation(
            task_type=TaskType.DETECTION,
            labels=[Label(key="class_label", value=annotation["class_label"])],
            bounding_box=BoundingBox.from_extrema(
                xmin=annotation["bbox"]["xmin"],
                xmax=annotation["bbox"]["xmax"],
                ymin=annotation["bbox"]["ymin"],
                ymax=annotation["bbox"]["ymax"],
            )
        )
        for annotation in element["annotations"]
        if len(annotation) > 0
    ]

    # create and return GroundTruth
    return GroundTruth(
        datum=datum,
        annotations=annotations,
    )

In [9]:
for element in object_detections:
    # create groundtruth
    groundtruth = create_groundtruth_from_object_detection_dict(element)

    # add groundtruth to dataset
    dataset.add_groundtruth(groundtruth)
    
    print(groundtruth)

{'datum': {'dataset': 'myDataset', 'uid': 'img3', 'metadata': {'path': 'a/b/c/img3.png'}, 'geospatial': {}}, 'annotations': [{'task_type': 'object-detection', 'labels': [{'key': 'class_label', 'value': 'dog', 'score': None}], 'metadata': {}, 'bounding_box': {'polygon': {'points': [{'x': 16.0, 'y': 130.0}, {'x': 70.0, 'y': 130.0}, {'x': 70.0, 'y': 150.0}, {'x': 16.0, 'y': 150.0}]}}, 'polygon': None, 'multipolygon': None, 'raster': None, 'jsonb': None}, {'task_type': 'object-detection', 'labels': [{'key': 'class_label', 'value': 'person', 'score': None}], 'metadata': {}, 'bounding_box': {'polygon': {'points': [{'x': 89.0, 'y': 10.0}, {'x': 97.0, 'y': 10.0}, {'x': 97.0, 'y': 110.0}, {'x': 89.0, 'y': 110.0}]}}, 'polygon': None, 'multipolygon': None, 'raster': None, 'jsonb': None}]}
{'datum': {'dataset': 'myDataset', 'uid': 'img4', 'metadata': {'path': 'a/b/c/img4.png'}, 'geospatial': {}}, 'annotations': [{'task_type': 'object-detection', 'labels': [{'key': 'class_label', 'value': 'cat', 's

/home/czaloom/velour/.env-velour/lib/python3.10/site-packages/velour/coretypes.py:808: UserWarning: GroundTruth for datum with uid `img5` contains no annotations.
  warnings.warn(


# **Image Semantic Segmentation**

In [10]:
image_segmentations = [
    {"path": "a/b/c/img6.png", "annotations": [{"class_label": "dog", "contour": [[{"x": 10.0, "y": 15.5}, {"x": 20.9, "y": 50.2}, {"x": 25.9, "y": 28.4}]]}]},
    {"path": "a/b/c/img7.png", "annotations": [{"class_label": "cat", "contour": [[{"x": 97.2, "y": 40.2}, {"x": 33.33, "y": 44.3}, {"x": 10.9, "y": 18.7}]]}]},
    {"path": "a/b/c/img8.png", "annotations": [{"class_label": "car", "contour": [[{"x": 10.0, "y": 15.5}, {"x": 20.9, "y": 50.2}, {"x": 25.9, "y": 28.4}], [{"x": 60.0, "y": 15.5}, {"x": 70.9, "y": 50.2}, {"x": 75.9, "y": 28.4}]]}]}
]

In [11]:
def create_groundtruth_from_image_segmentation_dict(element: dict):
    
    # create Datum using filename, save the full filepath into metadata
    datum = Datum(
        uid=Path(element["path"]).stem,
        metadata={
            "path": element["path"] 
        }
    )

    # create Annotations
    annotations = [
        Annotation(
            task_type=TaskType.SEGMENTATION,
            labels=[Label(key="class_label", value=annotation["class_label"])],
            polygon=Polygon(
                boundary=BasicPolygon(
                    points=[
                        Point(p["x"], p["y"])
                        for p in annotation["contour"][0]
                    ],
                ),
                holes=[
                    BasicPolygon(
                        points=[
                            Point(p["x"], p["y"])
                            for p in hole
                        ]
                    )
                    for hole in annotation["contour"][1:]
                ]
            )
        )
        for annotation in element["annotations"]
        if len(annotation["contour"]) > 0
    ]

    # create and return GroundTruth
    return GroundTruth(
        datum=datum,
        annotations=annotations,
    )

In [12]:
for element in image_segmentations:
    # create groundtruth
    groundtruth = create_groundtruth_from_image_segmentation_dict(element)

    # add groundtruth to dataset
    dataset.add_groundtruth(groundtruth)
    
    print(groundtruth)

{'datum': {'dataset': 'myDataset', 'uid': 'img6', 'metadata': {'path': 'a/b/c/img6.png'}, 'geospatial': {}}, 'annotations': [{'task_type': 'semantic-segmentation', 'labels': [{'key': 'class_label', 'value': 'dog', 'score': None}], 'metadata': {}, 'bounding_box': None, 'polygon': {'boundary': {'points': [{'x': 10.0, 'y': 15.5}, {'x': 20.9, 'y': 50.2}, {'x': 25.9, 'y': 28.4}]}, 'holes': []}, 'multipolygon': None, 'raster': None, 'jsonb': None}]}
{'datum': {'dataset': 'myDataset', 'uid': 'img7', 'metadata': {'path': 'a/b/c/img7.png'}, 'geospatial': {}}, 'annotations': [{'task_type': 'semantic-segmentation', 'labels': [{'key': 'class_label', 'value': 'cat', 'score': None}], 'metadata': {}, 'bounding_box': None, 'polygon': {'boundary': {'points': [{'x': 97.2, 'y': 40.2}, {'x': 33.33, 'y': 44.3}, {'x': 10.9, 'y': 18.7}]}, 'holes': []}, 'multipolygon': None, 'raster': None, 'jsonb': None}]}
{'datum': {'dataset': 'myDataset', 'uid': 'img8', 'metadata': {'path': 'a/b/c/img8.png'}, 'geospatial':

# **Text Classification**

In [13]:
text_classifications = [
    {"path": "a/b/c/text1.txt", "annotations": [{"sentiment": {"context": "Is the content of this product review postive?", "label": "positive"}}]}
]

In [14]:
def create_groundtruth_from_text_classification_dict(element: dict):
    
    # create Datum using filename, save the full filepath into metadata
    datum = Datum(
        uid=Path(element["path"]).stem,
        metadata={
            "path": element["path"],
            "context": element["annotations"][0]["sentiment"]["context"]
        }
    )

    # create Annotation
    annotations = [
        Annotation(
            task_type=TaskType.CLASSIFICATION,
            labels=[
                Label(
                    key="label", 
                    value=element["annotations"][0]["sentiment"]["label"]
                )
            ]
        )
    ]

    # create and return GroundTruth
    return GroundTruth(
        datum=datum,
        annotations=annotations,
    )

In [15]:
for element in text_classifications:
    # create groundtruth
    groundtruth = create_groundtruth_from_text_classification_dict(element)

    # add groundtruth to dataset
    dataset.add_groundtruth(groundtruth)
    
    print(groundtruth)

{'datum': {'dataset': 'myDataset', 'uid': 'text1', 'metadata': {'path': 'a/b/c/text1.txt', 'context': 'Is the content of this product review postive?'}, 'geospatial': {}}, 'annotations': [{'task_type': 'classification', 'labels': [{'key': 'label', 'value': 'positive', 'score': None}], 'metadata': {}, 'bounding_box': None, 'polygon': None, 'multipolygon': None, 'raster': None, 'jsonb': None}]}


# **Text Token Classification**

This example is nuanced because the way datums are communicated is different between the source dictionary and Velour.

Specifically, the location of the token is considered the `Datum` as it encodes what the subject of comparison is. We can apply this by encoding the positions into the `Datum` uid. As long as the model shares the same tokenizer as the dataset we can guarantee that this will lead to proper evaluation.

In [16]:
token_classifications = [
    {"path": "a/b/c/text2.text", "annotations": [{"token_classification": {"start_position": 19, "end_position": 22, "label": "place"}}]}
]

In [17]:
def create_groundtruth_from_token_classification_dict(element: dict):

    filename = Path(element["path"]).stem
    start_position = element["annotations"][0]["token_classification"]["start_position"]
    end_position = element["annotations"][0]["token_classification"]["end_position"]
    
    # create Datum using filename, save the full filepath into metadata
    datum = Datum(
        uid=f"{filename}_{start_position}_{end_position}",
        metadata={
            "path": element["path"],
            "filename": filename,
            "start_position": start_position,
            "end_position": end_position,
        }
    )

    # create Annotation
    annotations = [
        Annotation(
            task_type=TaskType.CLASSIFICATION,
            labels=[
                Label(
                    key="label", 
                    value=element["annotations"][0]["token_classification"]['label']
                )
            ],
        )
    ]

    # create and return GroundTruth
    return GroundTruth(
        datum=datum,
        annotations=annotations,
    )

In [18]:
for element in token_classifications:
    # create groundtruth
    groundtruth = create_groundtruth_from_token_classification_dict(element)

    # add groundtruth to dataset
    dataset.add_groundtruth(groundtruth)
    
    print(groundtruth)

{'datum': {'dataset': 'myDataset', 'uid': 'text2_19_22', 'metadata': {'path': 'a/b/c/text2.text', 'filename': 'text2', 'start_position': 19, 'end_position': 22}, 'geospatial': {}}, 'annotations': [{'task_type': 'classification', 'labels': [{'key': 'label', 'value': 'place', 'score': None}], 'metadata': {}, 'bounding_box': None, 'polygon': None, 'multipolygon': None, 'raster': None, 'jsonb': None}]}


# **Datatset Finalization**

Now that we are done creating `GroundTruth` objects we can move on to finalizing the `Dataset`. This is required for evaluation but not for `Model` creation so it could also be done later. Velour makes this a requirement for traceability reasons, you can be certain that a `Dataset` that was finalized weeks ago will still be the same today.



In [22]:
dataset.finalize()

<Response [200]>

# **Creating a Model**

In [23]:
model = Model(
    client=client,
    name="myModel",
    metadata={
        "foo": "bar",
        "some_number": 4321,
    },
    geospatial=None,
    reset=True,
)

# **Exploring the Client**

In [28]:
client.get_datasets()

[{'id': 1,
  'name': 'coco2017-panoptic',
  'metadata': {'url': 'http://cocodataset.org',
   'year': 2018.0,
   'version': '1.0',
   'licenses': "[{'url': 'http://creativecommons.org/licenses/by-nc-sa/2.0/', 'id': 1, 'name': 'Attribution-NonCommercial-ShareAlike License'}, {'url': 'http://creativecommons.org/licenses/by-nc/2.0/', 'id': 2, 'name': 'Attribution-NonCommercial License'}, {'url': 'http://creativecommons.org/licenses/by-nc-nd/2.0/', 'id': 3, 'name': 'Attribution-NonCommercial-NoDerivs License'}, {'url': 'http://creativecommons.org/licenses/by/2.0/', 'id': 4, 'name': 'Attribution License'}, {'url': 'http://creativecommons.org/licenses/by-sa/2.0/', 'id': 5, 'name': 'Attribution-ShareAlike License'}, {'url': 'http://creativecommons.org/licenses/by-nd/2.0/', 'id': 6, 'name': 'Attribution-NoDerivs License'}, {'url': 'http://flickr.com/commons/usage/', 'id': 7, 'name': 'No known copyright restrictions'}, {'url': 'http://www.usa.gov/copyright.shtml', 'id': 8, 'name': 'United States

In [29]:
client.get_models()

[{'id': 1, 'name': 'yolov8n-seg', 'metadata': {}, 'geospatial': {}},
 {'id': 2,
  'name': 'myModel',
  'metadata': {'foo': 'bar', 'some_number': 4321.0},
  'geospatial': {}}]

# **Exploring the Dataset**

`Dataset.get_datums()`

In [19]:
for datum in dataset.get_datums():
    print(datum)

{'dataset': 'myDataset', 'uid': 'img1', 'metadata': {'path': 'a/b/c/img1.png'}, 'geospatial': {}}
{'dataset': 'myDataset', 'uid': 'img2', 'metadata': {'path': 'a/b/c/img2.png'}, 'geospatial': {}}
{'dataset': 'myDataset', 'uid': 'img3', 'metadata': {'path': 'a/b/c/img3.png'}, 'geospatial': {}}
{'dataset': 'myDataset', 'uid': 'img4', 'metadata': {'path': 'a/b/c/img4.png'}, 'geospatial': {}}
{'dataset': 'myDataset', 'uid': 'img5', 'metadata': {'path': 'a/b/c/img5.png'}, 'geospatial': {}}
{'dataset': 'myDataset', 'uid': 'img6', 'metadata': {'path': 'a/b/c/img6.png'}, 'geospatial': {}}
{'dataset': 'myDataset', 'uid': 'img7', 'metadata': {'path': 'a/b/c/img7.png'}, 'geospatial': {}}
{'dataset': 'myDataset', 'uid': 'img8', 'metadata': {'path': 'a/b/c/img8.png'}, 'geospatial': {}}
{'dataset': 'myDataset', 'uid': 'text1', 'metadata': {'path': 'a/b/c/text1.txt', 'context': 'Is the content of this product review postive?'}, 'geospatial': {}}
{'dataset': 'myDataset', 'uid': 'text2_19_22', 'metadat

`Dataset.get_groundtruth(uid: str)`

In [20]:
for datum in dataset.get_datums():
    groundtruth = dataset.get_groundtruth(datum.uid)
    print(groundtruth)

{'datum': {'dataset': 'myDataset', 'uid': 'img1', 'metadata': {'path': 'a/b/c/img1.png'}, 'geospatial': {}}, 'annotations': [{'task_type': 'classification', 'labels': [{'key': 'class_label', 'value': 'dog', 'score': None}], 'metadata': {}, 'bounding_box': None, 'polygon': None, 'multipolygon': None, 'raster': None, 'jsonb': None}]}
{'datum': {'dataset': 'myDataset', 'uid': 'img2', 'metadata': {'path': 'a/b/c/img2.png'}, 'geospatial': {}}, 'annotations': [{'task_type': 'classification', 'labels': [{'key': 'class_label', 'value': 'cat', 'score': None}], 'metadata': {}, 'bounding_box': None, 'polygon': None, 'multipolygon': None, 'raster': None, 'jsonb': None}]}
{'datum': {'dataset': 'myDataset', 'uid': 'img3', 'metadata': {'path': 'a/b/c/img3.png'}, 'geospatial': {}}, 'annotations': [{'task_type': 'object-detection', 'labels': [{'key': 'class_label', 'value': 'dog', 'score': None}], 'metadata': {}, 'bounding_box': {'polygon': {'points': [{'x': 16.0, 'y': 130.0}, {'x': 70.0, 'y': 130.0}, 

`Dataset.get_labels()`

In [21]:
for label in dataset.get_labels():
    print(label)

{'key': 'class_label', 'value': 'car', 'score': None}
{'key': 'label', 'value': 'place', 'score': None}
{'key': 'class_label', 'value': 'cat', 'score': None}
{'key': 'class_label', 'value': 'dog', 'score': None}
{'key': 'label', 'value': 'positive', 'score': None}
{'key': 'class_label', 'value': 'person', 'score': None}
